In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy as sp
from scipy import integrate
from scipy.interpolate import interp1d 
from classy import Class
import math as math
%matplotlib inline
import sys, platform, os
import re
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
import camb
from camb import model, initialpower

In [4]:
#Fiducial cosmological parameters
c=3e5
hubble=0.678
omegab=0.022*pow(hubble,-2)
omegac=0.119*pow(hubble,-2)
om0=omegac+omegab
H00=100*hubble
Ass=2.14e-9
nss = 0.968
gamma=0.545

#table 1 from P&Y 1807.07076
euclid_data = np.loadtxt('tabula-1807.07076v2 porciani.txt')

#table 1 from P&Y has columns z, V, ng, b1, b2, bs, M
z_euclid = euclid_data[:,0]
v_euclid =  interp1d(z_euclid,10.0 * (1.0/(hubble**3)) * 1e9 *euclid_data[:,1])
ng_euclid = interp1d(z_euclid, 1e-3 * hubble**3 * euclid_data[:,2])
b1_euclid = interp1d(z_euclid, euclid_data[:,3])
b2_euclid = interp1d(z_euclid, euclid_data[:,4])
bs_euclid = interp1d(z_euclid, euclid_data[:,5])

#class, want to get the stuff all from camb though
cosmo = Class()
cosmo.compute()
background = cosmo.get_background()

#Set up the fiducial cosmology (CAMB)
pars = camb.CAMBparams()
#Set cosmology
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

#calculate results for these parameters
results = camb.get_results(pars)
background = camb.get_background(pars)

#Get matter power spectrum at z=zbin

zbin = 0.0
pars.set_matter_power(redshifts=[zbin], kmax=2.0)
#Use linear modelling
pars.NonLinear = model.NonLinear_none

#Non-Linear spectra (Halofit)
#pars.NonLinear = model.NonLinear_both

results = camb.get_results(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=2.0, npoints = 10000)
s8 = np.array(results.get_sigma8())

#Pm in Mpc^3 units, at z=0
Pmz0=interp1d(kh*hubble, (pk[0]/pow(hubble,3)))


#Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0+om0*pow(1+zc,3))
#Define the comoving distance
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return sp.integrate.romberg(drdz,0,zc)

#Define the growth function in LCDM
def get_growth(zz):
    omz=om0*pow(1+zz,3)/(om0*pow(1+zz,3)+1-om0)
    return pow(omz,gamma)

#Get the growth factor 
def Dg_dz(zz):
    return get_growth(zz)/(1+zz)
def Dgz(zc):
    ans = sp.integrate.romberg(Dg_dz, 0.0, zc)
    return np.exp(-ans)

#Power spectrum as fn of z 
def Pmz(kk,zc):    
    return pow(Dgz(zc),2)*Pmz0(kk)

In [26]:
Z=1.0

Hu = cosmo.Hubble(Z)*(1/(1+Z))
H0 = cosmo.Hubble(0)
om_m0 = 0.308
om_m = om_m0 * (H0**2/Hu**2) *(1+Z)
f = cosmo.scale_independent_growth_factor_f(Z)
df = Hu* ((1/2)*(3*om_m -4)*f - f**2 + (3/2)*om_m)
dHu = H0**2 * (-(1/2)* (1+Z) * om_m0 + (1/(1+Z))**2 * (1-om_m0))
ddHu = H0**2 * ( (1/2)*Hu*(1+Z)*om_m0 + (1/(1+Z))**2 * 2 * Hu * (1-om_m0) )
chi = cosmo.angular_distance(Z)*(1+Z)
cap_L = 1
partdQ=0
B1 = 0.9 + 0.4 * Z
db1 = -0.4 * Hu * (1 + Z)
B2 = -0.704172 - 0.207993 * Z + 0.183023 * Z**2 - 0.00771288 * Z**3
b_e = -4
db_e=0
Q=(-1.0)*(-0.66 - 2.95 * Z + 1.59 * Z**2 - 0.40 * Z**3 + 0.04 * Z**4)
dQ= 0
gamma1 = Hu* (f * (b_e - 2*Q -(2*(1-Q)/(chi*Hu)) - (dHu/Hu**2)))
gamma2 = Hu**2 * (f*(3-b_e) + (3/2)*om_m*(2+b_e - f- 4*Q - (2*(1-Q)/(chi*Hu)) - (dHu/Hu**2) ))
partdb1 = 0 
fnl=0

#beta coeffs
beta = {}
beta[1] = Hu**4 * ((9.0/4.0)*om_m**2 * (6- 2*f * (2*b_e - 4*Q - (4*(1-Q))/(chi*Hu) - (2*dHu)/(Hu**2)) - (2*df)/(Hu) + b_e**2 + 5*b_e - 8*b_e*Q + 4*Q + 16*Q**2 - 16*partdQ - 8*dQ/Hu + db_e/Hu + (2/(chi**2 * Hu**2))*(1-Q + 2*Q**2 - 2*partdQ)  - (2/(chi*Hu))*(3+2*b_e - 2*b_e*Q - 3*Q + 8*Q**2 - (3*dHu/Hu**2)*(1-Q) - 8*partdQ - 2*dQ/Hu) + (dHu/Hu**2)*(-7 -2*b_e + 8*Q + (3*dHu/Hu**2)) - (ddHu/Hu**3))  + ((3/2)*om_m * f)*(5 - 2*f*(4-b_e) + (2*df/Hu) + 2*b_e*(5 + ((2*(1-Q))/(chi*Hu))) - (2*db_e/Hu) - 2*b_e**2 + 8*b_e*Q - 28*Q - (14*(1-Q)/(chi*Hu)) - 3*dHu/Hu**2 + 4*(2-(1/(chi*Hu)))*(dQ/Hu) ) + ((3/2)*om_m*f**2)*(-2 + 2*f -b_e + 4*Q + (2*(1-Q)/(chi*Hu)) + (3*dHu/Hu**2) - (12/3)*fnl ) + f**2 * (12-7*b_e + b_e**2 + (db_e/Hu) + (b_e - 3)*(dHu/Hu**2)) - (3/2)*om_m*(df/Hu))
beta[2] = Hu**4* ((9/2)*om_m**2 * (-1 + b_e - 2*Q - ((2*(1-Q))/(chi*Hu)) - (dHu/Hu**2) ) + 3*om_m*f*(-1 + 2*f -b_e + 4*Q + (2*(1-Q)/(chi*Hu)) + (3*dHu/Hu**2) )  + 3*om_m * f**2 * (-1+b_e-2*Q - ((2*(1-Q))/(chi*Hu)) - (dHu/Hu**2))  + 3*om_m*(df/Hu))
beta[3] = Hu**3 * ((9/4)*om_m**2 *(f-2+2*Q) + (3/2)*om_m*f * (-2 -f*(-3 + f + 2*b_e - 3*Q - ((4*(1-Q))/(chi*Hu)) - (2*dHu/Hu**2)) - (df/Hu) + 3*b_e + b_e**2 - 6*b_e*Q + 4*Q + 8*Q**2 - 8*partdQ - 6*(dQ/Hu) + (db_e/Hu) + (2/(chi**2*Hu**2))*(1-Q +2*Q**2 - 2*partdQ) + (2/(chi*Hu))*(-1 -2*b_e + 2*b_e*Q + Q - 6*Q**2 + (3*dHu/Hu**2)*(1-Q) + 6*partdQ + 2*(dQ/Hu) ) - (dHu/Hu**2)*(3+2*b_e - 6*Q - (3*dHu/Hu**2)) - (ddHu/Hu**3))  + f**2 * (-3 + 2*b_e*(2+ ((1-Q)/(chi*Hu))) - b_e**2 + 2*b_e*Q - 6*Q - (db_e/Hu) - ((6*(1-Q))/(chi*Hu)) + 2*(1-(1/(chi*Hu)))*(dQ/Hu) ))
beta[4] = (Hu**3 * ((9/2)*om_m*f * (-b_e + 2*Q +  (2*(1-Q)/(chi*Hu)) + (dHu/Hu**2))))
beta[5] = (Hu**3 * ( 3* om_m * f * (b_e - 2*Q -(2*(1-Q)/(chi*Hu)) - (dHu/Hu**2))  ))
beta[6] = Hu**2 * ((3/2) *om_m * (2-2*f + b_e - 4*Q - ((2*(1-Q))/(chi*Hu)) - (dHu/Hu**2)))
beta[7] = Hu**2*(f*(3-b_e))
beta[8]= Hu**2*(3*om_m*f*(2-f-2*Q) + f**2 * (4 + b_e - b_e**2 + 4*b_e*Q - 6*Q - 4*Q**2 + 4*partdQ + 4*(dQ/Hu) -(db_e/Hu) - (2/(chi**2 * Hu**2))*(1-Q+2*Q**2 - 2*partdQ) - (2/(chi*Hu))*(3-2*b_e + 2*b_e*Q - Q - 4*Q**2 + ((3*dHu)/(Hu**2))*(1-Q) + 4*partdQ + 2*(dQ/Hu)) - (dHu/Hu**2)*(3-2*b_e+4*Q+((3*dHu)/(Hu**2))) + (ddHu/Hu**3)))
beta[9] = (Hu**2 * ( -(9/2)*om_m*f ))
beta[10] = (Hu**2 * (3 * om_m * f))
beta[11] = Hu**2 * (3*om_m*(1+f) + 2*f - f**2*(-1+b_e-2*Q - ((2*(1+Q))/(chi*Hu)) - (dHu/Hu**2)))
beta[12] = Hu**2*((3/2)*om_m * (-2 + B1 * (2+b_e-4*Q - ((2*(1-Q))/(chi*Hu)) - (dHu/Hu**2)) + (db1/Hu) + 2*(2-(1/(chi*Hu)))*partdb1) - f*(2+ B1*(f-3+b_e) + (db1/Hu)))
beta[13] = ( (9.0/4.0)*om_m**2.0*Hu**2.0 )+ ( 1.5*om_m*f*Hu**2.0*(-(2.0*f)+(2.0*b_e)-(6.0*Q) -((4.0*(1.0-Q))/(chi*Hu)) -((3.0*dHu)/Hu**2.0) ) )+ ( f**2.0*Hu**2.0*(5.0-b_e) )
beta[14] = Hu* ( -(3/2)*om_m*B1 )
beta[15] = Hu*2*f**2 
beta[16] = Hu*(f*(B1 * (f+b_e-2*Q - ((2*(1-Q))/(chi*Hu)) - (dHu/Hu**2)) + (db1/Hu) + 2*(1- (1/(chi*Hu)))*partdb1 ))
beta[17] = Hu*(-(3/2)*om_m*f)
beta[18] = Hu* ( (3/2)*om_m*f - f**2 * (3 - 2*b_e + 4*Q + ((4*(1-Q))/(chi*Hu)) + (3*dHu/Hu**2)) )
beta[19] = Hu * (f* (b_e - 2*Q - ((2*(1-Q))/(chi*Hu)) - (dHu/Hu**2)))



def Pm(i):
    kk = k[i]
    return Pmz(kk,Z)

In [42]:
#k = {1:k1,2:k2,3:k3,"theta":theta}
#mu = {1:mu1,2:mu2,3:mu3}

# def get_mus(self,MU_1,PHI,k):
#     mu = {1:MU_1}
#     mu[2]=mu[1]*np.cos(k["theta"]) + np.sqrt(1.0-mu[1]**2) * np.sin(k["theta"])*np.cos(PHI)
#     mu[3] = - (k[1] / k[3]) * mu[1] - (k[2] / k[3]) * mu[2]
#     return mu
class NotATriangle(Exception):
    pass

def get_theta(k1,k2,k3):
    x = 0.5 * (k3**2 - (k1**2 + k2**2))/(k1 * k2)
    if x>1 or x<-1:
        raise NotATriangle()
    return np.arccos(x)

def get_mus(MU_1,PHI,k):
    mu = {1:MU_1}
    mu[2]=mu[1]*np.cos(k["theta"]) + np.sqrt(1.0-mu[1]**2) * np.sin(k["theta"])*np.cos(PHI)
    mu[3] = - (k[1] / k[3]) * mu[1] - (k[2] / k[3]) * mu[2]
    return mu


def get_cosinus(i,j,k):
    perms = [1,2,3]
    assert i!=j
    assert i in perms
    assert j in perms
    perms.remove(i)
    perms.remove(j)
    l = perms[0]
    
    return 0.5 * (k[l]**2 - k[i]**2 - k[j]**2) / (k[i] + k[j])
    

def E(i,j,l,k):
    cosinus = get_cosinus(i,j,k)
    return ((k[i]**2 * k[j]**2) / (k[l]) ) * (3 + 2 * cosinus * (k[i]/k[j] + k[j]/k[i]) + cosinus**2  )

def F(i,j,l,k):
    cosinus = get_cosinus(i,j,k)
    return 10/7 + cosinus * (k[i]/k[j] + k[j]/k[i]) + (1 - 3/7)*cosinus**2
    
def G(i,j,l,k):
    cosinus = get_cosinus(i,j,k)
    return 6/7 + cosinus * (k[i]/k[j] + k[j]/k[i]) + (2 - 6/7) * cosinus**2 

def KN1(i,mu,B1,f):
    return B1 + f * mu[i]**2

def KGR1(i,mu,gamma1,gamma2):
    return (gamma2/k[i])**2 + (gamma1/k[i])*1j*mu[i]


#this is where Eline's gonna fuck up - she did but Reese fixed :P
def KN2(i,j,l,k,mu,B1,B2,f):
    cosinus = get_cosinus(i,j,k)
    F_f = F(i,j,l,k)
    G_f = G(i,j,l,k)
    p1 = B1 * F_f + B2 + f * G_f + mu[l]**2 
    p2 = - 4/7 * (B1 - 1) * (cosinus**2 - 1/3)
    p3 = f**2 * (mu[i] * mu[j])/(k[i] * k[j]) * (mu[i] * k[i] + mu[j] * k[j])**2
    p4 = B1 * f/(k[i] * k[j]) * ((mu[i]**2 + mu[j]**2) * k[i] * k[j] + mu[i] * mu[j] * (k[i]**2 + k[j]**2))
    return p1 + p2 + p3 + p4 


def KGR2(i,j,l,k,mu,beta):
    cosinus = get_cosinus(i,j,k)
    E_f = E(i,j,l,k)
    F_f = F(i,j,l,k)
    G_f = G(i,j,l,k)
    k_prod = k[i]**2 * k[j]**2
    p1 = beta[1] + E_f * beta[2] 
    p2 = 1j* ((mu[i] * k[i] + mu[j] * k[j]) * beta[3] + mu[l] * k[l] * (beta[4] + E_f * beta[5]) )
    p3 = k_prod/(k[l]**2) * (F_f * beta[6] + G_f * beta[7]) + (mu[i] * k[i] * mu[j] *k[j])*beta[8]
    p4 = mu[l]**2 * k[l]**2 * (beta[9] + E_f * beta[10]) + (k[i] * k[j] * cosinus) * beta[11]
    p5 = (k[i]**2 + k[j]**2) * beta[12] + (mu[i]**2 * k[i]**2 + mu[j]**2 + k[j]**2) * beta[13] 
    p_comp1 = (mu[i] * k[i]**3 + mu[j] * k[j]**3) * beta[14]
    p_comp2 = (mu[i] * k[i] + mu[j] * k[j]) * k[i] * k[j] * cosinus * beta[15]
    p_comp3 = k[i] * k[j] * (mu[i] * k[j] + mu[j] * k[i]) * beta[16]
    p_comp4 = (mu[i]**3 * k[i]**3 + mu[j]**3 * k[j]**3) * beta[17]
    p_comp5 = mu[i] * mu[j] * k[i] * k[j] * (mu[i] * k[i] + mu[j] * k[j]) * beta[18]
    p_comp6 = mu[l] * k_prod/k[l] * G_f * beta[19]
    real = p1 + p2 + p3 + p4 + p5
    comp = p_comp1 + p_comp2 + p_comp3 + p_comp4 + p_comp5 + p_comp6
    return  (1/k_prod) * (real + 1j * comp)


def B_perm(i,j,l,k,mu,B1,B2,gamma1,gamm2,beta,f):
    f1 = KN1(i,mu,B1,f)+KGR1(i,mu,gamma1,gamma2)
    f2 = KN1(j,mu,B1,f)+KGR1(j,mu,gamma1,gamma2)
    f3 = KN2(1,2,3,k,mu,B1,B2,f) + KGR2(i,j,l,k,mu,beta)
    return f1 * f2 * f3 * Pm(i) * Pm(j)

def B_full(k,mu,B1,B2,gamma1,gamma2,beta,f):
    return  B_perm(1,2,3,k,mu,B1,B2,gamma1,gamma2,beta,f) +\
            B_perm(1,3,2,k,mu,B1,B2,gamma1,gamma2,beta,f) +\
            B_perm(2,3,1,k,mu,B1,B2,gamma1,gamma2,beta,f)

In [47]:

k1_bins = np.arange(0.1,1,1/20)
k2_bins = np.arange(0.1,1,1/20)
k3_bins = np.arange(0.1,1,1/20)


for k1 in k1_bins:
    for k2 in k2_bins[k2_bins<=k1]:
        for k3 in k3_bins[k3_bins<=k2]:
            try:
                k = {1:k1,2:k2,3:k3,"theta":get_theta(k1,k2,k3)}
                mu = get_mus(-1,0.9,k)
                bisp = B_full(k,mu,B1,B2,gamma1,gamma2,beta,f)  
                print(abs(bisp)/1e9)
            except NotATriangle:
                continue
            

0.32360341260578573
0.2233366983942077
0.10568677334759871
0.073313754756612
0.09931854669294503
0.053572286739604326
0.05022510412400717
0.030798576926259882
0.0243965379694462
0.04446872292354421
0.053654932613100406
0.026754929875830724
0.018534083378398944
0.028101252059186718
0.016196089907886922
0.011790832885764546
0.009986741897456684
0.03887249677432023
0.06953240599243618
0.02569769468211358
0.015257604796884998
0.03219965651426402
0.015324667472511051
0.010015234143086095
0.007784845052575996
0.017196731549426518
0.009522935998415498
0.006615703740295623
0.005265621967528554
0.004636336018012392
0.02645206734875706
0.013146517449800088
0.016089520303069593
0.009076570072505011
0.006428543931912733
0.02065890501878776
0.009504867204775898
0.005997193253321045
0.004453157513066279
0.00369063345749373
0.011170361162046453
0.006018104611781912
0.004051091395655442
0.003107471814899659
0.002608697785877973
0.002362645460478843
0.01812376128752698
0.008624688855454888
0.0055070546

0.0002532363259269894
0.0001899710611975734
0.0001531542010584514
0.0001304944995378975
0.00011619008975609741
0.00010719786676225793
0.00202359845807971
0.0007870231282867103
0.00042016013566763976
0.0002678491580113163
0.00019160810954756325
0.00014874534305633877
0.00012273074761666555
0.0001061596062705057
9.537719727443877e-05
8.838316547698903e-05
8.406323970789513e-05
0.003810974489605114
0.001012930364479303
0.00046801114191583233
0.00027877643762779306
0.00019092346549298757
0.00014333984872742518
0.0001150440653737852
9.715363764743863e-05
8.540109628159293e-05
7.756744932167723e-05
7.237741553846947e-05
6.909839447396603e-05
6.729309956810907e-05
0.0013137307429152208
0.0005061011631990445
0.0002821092578721602
0.000186684772387594
0.000136398729068188
0.00010688308473902676
8.838614808059385e-05
7.627146286714673e-05
6.811922280389141e-05
6.259833515870371e-05
5.890333084252085e-05
5.6558169923482684e-05
5.527129965085411e-05
5.486548438709137e-05
0.0005934274789246322
0.00